激活函数。  
relu，与0和自己做max，最简单的非线性函数  
主要提供非线性性，早期nn解决不了异或问题，故引入

silu = x * sigmoid x
更光滑，处处可微
常称为Swish

### MLP层的改进

数学表示： 
GLU(x, W1, W2) = σ(W1x) ⊙ W2x, 
 
中间是Hadamard product，，σ(W1x)相当于门控，决定哪些权重更重要  
比MLP表达力更强，（多一倍的参数）


将σ改成swish，就得到了SwiGLU。目前常用的FFN层。  


FFN(x) = SwiGLU(x, W, W, W) = W(SiLU(Wx) ⊙ Wx)，

In [ ]:
def run_swiglu(
    d_model: int,
    d_ff: int,
    w1_weight: Float[Tensor, " d_ff d_model"],
    w2_weight: Float[Tensor, " d_model d_ff"],
    w3_weight: Float[Tensor, " d_ff d_model"],
    in_features: Float[Tensor, " ... d_model"],
) -> Float[Tensor, " ... d_model"]:

参数:
        d_model (int): 前馈输入和输出的维度。
        d_ff (int): SwiGLU内部上投影的维度。
        w1_weight (Float[Tensor, "d_ff d_model"]): W1的存储权重
        w2_weight (Float[Tensor, "d_model d_ff"]): W2的存储权重
        w3_weight (Float[Tensor, "d_ff d_model"]): W3的存储权重
        in_features (Float[Tensor, "... d_model"]): 前馈层的输入嵌入。

    返回:
        Float[Tensor, "... d_model"]: 与输入嵌入形状相同的输出嵌入。



x： ...,d_model

W1x = ...,d_ff

W3x = ...,d_ff

W1x ⊙ W3x = ...,d_ff

W2  = ...,d_model,d_ff
out = W2(W1x ⊙ W3x).    ... ,d_model


In [ ]:
# 可以使用的模块：为了数值稳定性，可以使用 torch.sigmoid()

class SwiGLU(nn.Module):
    def __init__(self, d_model, d_ff, device=None, dtype=None):
        super().__init__()
        self.d_model = d_model
        # 移除不必要的d_ff调整，直接使用传入的d_ff
        self.d_ff = d_ff
        self.w1 = Linear(d_model, self.d_ff)
        self.w2 = Linear(self.d_ff, d_model)
        self.w3 = Linear(d_model, self.d_ff)

    def load_weights(self, w1: torch.Tensor, w2: torch.Tensor, w3: torch.Tensor):
        """加载权重，确保形状匹配"""
        with torch.no_grad():
            # 直接复制权重，不需要转置（因为Linear类已经处理了）
            self.w1.weight.data = w1
            self.w2.weight.data = w2
            self.w3.weight.data = w3

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """
        SwiGLU前向传播：
        SwiGLU(x) = (SiLU(xW1) ⊙ xW3)W2
        """
        w1_out = self.w1(x)        # x @ W1
        w3_out = self.w3(x)        # x @ W3
        silu_out = w1_out * torch.sigmoid(w1_out)  # SiLU(xW1)
        gated = silu_out * w3_out   # SiLU(xW1) ⊙ xW3
        return self.w2(gated)       # (SiLU(xW1) ⊙ xW3)W2